In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Initialize variables
min_contour_width = 40
min_contour_height = 40
line_height = 550
offset = 10
matches = []
vehicles = 0

In [ ]:
# Initialize video capture (e.g., for a video file or webcam)
cap = cv2.VideoCapture("Video.mp4")  # Or 0 for webcam

In [ ]:
# Read the first two frames
ret, frame1 = cap.read()
ret, frame2 = cap.read()

In [ ]:
# Function to get centroid of contour
def get_centrolid(x, y, w, h):
    return (int(x + w / 2), int(y + h / 2))

In [ ]:


while ret:
    # Compute absolute difference between the frames
    d = cv2.absdiff(frame1, frame2)
    grey = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (5, 5), 0)
    ret, th = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    
    # Dilation and morphological closing to fill gaps
    dilated = cv2.dilate(th, np.ones((3, 3)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
    closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Iterate over contours to find valid ones
    for (i, c) in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(c)
        contour_valid = (w >= min_contour_width) and (h >= min_contour_height)

        if not contour_valid:
            continue
        
        # Draw bounding box
        cv2.rectangle(frame1, (x-10, y-10), (x+w+10, y+h+10), (255, 0, 0), 2)
        
        # Draw line for counting vehicles
        cv2.line(frame1, (0, line_height), (1200, line_height), (0, 255, 0), 2)
        
        # Get centroid of the contour
        centroid = get_centrolid(x, y, w, h)
        matches.append(centroid)
        
        # Draw centroid point
        cv2.circle(frame1, centroid, 5, (0, 255, 0), -1)
        
        # Count vehicles crossing the line
        for (x, y) in matches:
            if y < (line_height + offset) and y > (line_height - offset):
                vehicles += 1
                matches.remove((x, y))
    
    # Put total vehicles detected text on frame
    cv2.putText(frame1, "Total Vehicle Detected: " + str(vehicles), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 170, 0), 2)

    # Convert the frame to RGB for displaying in Jupyter
    frame1_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
    
    # Display the image inline
    plt.imshow(frame1_rgb)
    plt.axis('off')  # Hide axes
    plt.show()

    # Update frames for the next iteration
    frame1 = frame2
    ret, frame2 = cap.read()

cv2.destroyAllWindows()
cap.release()